For each set of NEs predicted by each package (i.e. Stanza and Spacy), how many of them can be found mentioned in the KG Graph you ollected for the person?7 

return for each package (i.e. Stanza and Spacy), the ratio of
predicted NEs that you can be confidently said to be in the KG
graph for the person.

7 note that you should take into consid-
eration the NE span in the text may not
be an exact match with KG entity label.
Here you can use regular expressions
to match the NEs found in the text
with the RDF entities present in the
associated texts.

In [22]:
import os 
import pandas as pd 
import numpy as np
import json

In [7]:
par_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(par_dir, '2.2 NER - analysis by entity type')

stanza_dir = os.path.join(data_dir, "df_entities_spacy_processed.csv")
spacy_dir = os.path.join(data_dir, "df_entities_stanza_processed.csv")

entities_stanza = pd.read_csv(stanza_dir)
entities_spacy = pd.read_csv(spacy_dir)

In [8]:
list_people = list(set(list(np.unique(entities_spacy["subject"])) + list(np.unique(entities_stanza["subject"]))))
list_entities = list(set(list(np.unique(entities_spacy["label"])) + list(np.unique(entities_stanza["label"]))))

In [25]:
for person in list_people:
    
    # filtering each df by the subject of the bio
    filter_df_spacy = entities_spacy[entities_spacy["subject"] == person]
    filter_df_stanza = entities_stanza[entities_stanza["subject"] == person]
    
    # getting the category to know which Data Directory subfolder we have to access
    category = filter_df_spacy["category"].iloc[0]
    
    # print(category)
    # getting the entities found in the bios per package
    set_entities_spacy = set(filter_df_spacy["text"].to_list())
    set_entities_stanza = set(filter_df_stanza["text"].to_list())
    
    # total amount of unique entities
    total_entities_spacy = len(set_entities_spacy)
    total_entities_stanza = len(set_entities_stanza)
    
    found_entities_spacy = 0
    found_entities_stanza = 0   
    
    # loading the json graphs to query them
    par_dir = os.path.dirname(os.path.dirname(os.getcwd()))
    name_file_graph = person.replace(" ", "__") + ".json"
    data_dir = os.path.join(par_dir, "Data Directory")
    
    if category == "Chemistry":
        graph_dir = os.path.join(data_dir, "chemistry_nobel_laureate")
    
    else:
        graph_dir = os.path.join(data_dir, "physics_nobel_laureate")
        
    person_graph_path = os.path.join(graph_dir, name_file_graph)
    # print(person_graph_path)
    
    with open(person_graph_path, "r") as f:
        personal_graph = json.load(f)
    # if f:
    #     print("Graph load successfully!")
    
    
    
    